In [15]:
import splitfolders  # or import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/home/kritika/Downloads/FYP/dataset/GTSRB/mix_data/", output="/home/kritika/Downloads/FYP/dataset/GTSRB/split_data/", seed=1337, ratio=(.7, .2, .1), group_prefix=None)

Copying files: 32675 files [02:12, 246.94 files/s] 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [5]:
from PIL import Image
import matplotlib.pyplot as plt
from skimage import exposure
from skimage import transform
import numpy as np
import os

data = []
labels = []
path = "/home/kritika/Downloads/FYP/dataset/GTSRB/mix_data/"

for folder in range(43):
    for img in os.listdir(path+str(folder)):
        image = Image.open(path+str(folder)+"/"+img)
        if image.mode == "RGBA":
            image = image.convert("RGB")
        image = np.array(image)
        image = transform.resize(image, (32, 32))
        # update the list of data and labels, respectively
        data.append(image)
        labels.append(int(folder))
        
data = np.array(data)
labels = np.array(labels)

In [6]:
print(data.shape,labels.shape)

(32675, 32, 32, 3) (32675,)


In [13]:
np.save('data.npy', data)
np.save('labels.npy', labels)

In [16]:
X_train, X_test, y_train,y_test = train_test_split(data,labels,test_size=0.25,random_state=42, stratify = labels)

print(X_train.shape, X_test.shape, y_train.shape,y_test.shape)

y_train = to_categorical(y_train,43)
y_test = to_categorical(y_test,43)

(24506, 32, 32, 3) (8169, 32, 32, 3) (24506,) (8169,)


In [3]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compiling the model with adam optimizer
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)         

In [18]:
epochs = 20

history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_test,y_test))

Epoch 1/20
383/383 [==============================] - 81s 206ms/step - loss: 3.2551 - accuracy: 0.1381 - val_loss: 1.3899 - val_accuracy: 0.6337
Epoch 2/20
383/383 [==============================] - 74s 192ms/step - loss: 1.4835 - accuracy: 0.5857 - val_loss: 0.9511 - val_accuracy: 0.7465
Epoch 3/20
383/383 [==============================] - 74s 194ms/step - loss: 1.1142 - accuracy: 0.6848 - val_loss: 0.7719 - val_accuracy: 0.7837
Epoch 4/20
383/383 [==============================] - 74s 192ms/step - loss: 0.9619 - accuracy: 0.7247 - val_loss: 0.6938 - val_accuracy: 0.8013
Epoch 5/20
383/383 [==============================] - 74s 194ms/step - loss: 0.8684 - accuracy: 0.7430 - val_loss: 0.6577 - val_accuracy: 0.8139
Epoch 6/20
383/383 [==============================] - 74s 193ms/step - loss: 0.7817 - accuracy: 0.7728 - val_loss: 0.6282 - val_accuracy: 0.8183
Epoch 7/20
383/383 [==============================] - 74s 193ms/step - loss: 0.7152 - accuracy: 0.7869 - val_loss: 0.5655 - val_ac

In [19]:
model.save('traffic_classifier.h5')